In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\pavan\OneDrive\Desktop\prep\datasets\creditcard\creditcard.csv")

In [2]:
df.shape

(284807, 31)

In [3]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [4]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

Imbalanced dataset because the number of 1's here is in hundered's range where as 0's is in some thousand's range.

In [5]:
X = df.drop("Class", axis = 1)
y = df['Class']

#### Cross validation like KFOLD and hyperparameter Tuning

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV

In [7]:
#just to what values it would give, which is used in next line 
10.0 **np.arange(-2,3)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])

In [8]:
log_class=LogisticRegression()
grid={'C':10.0 **np.arange(-2,3),'penalty':['l1','l2']}
cv=KFold(n_splits=5,random_state=None,shuffle=False)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7)

In [10]:
clf=GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

C:\Users\pavan\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\pavan\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pavan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\pavan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise ValueError(
Value

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [12]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85255    38]
 [   44   106]]
0.9990402958697612
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85293
           1       0.74      0.71      0.72       150

    accuracy                           1.00     85443
   macro avg       0.87      0.85      0.86     85443
weighted avg       1.00      1.00      1.00     85443



In [15]:
#class weights - we are trying to give more importance to less frequently used data using this
class_weight = ({0: 1, 1:100})

In [17]:
from sklearn.ensemble import RandomForestClassifier 

classifier= RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [18]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85287     6]
 [   34   116]]
0.9995318516437859
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85293
           1       0.95      0.77      0.85       150

    accuracy                           1.00     85443
   macro avg       0.98      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443



## Different ways to handle

### Under Sampling

Try to reduce points of maximum labels.
For example, for class feature we have 0's in thousands range which we want to reduce



In [21]:
from collections import Counter
Counter(y_train)

Counter({0: 199022, 1: 342})

In [34]:
from imblearn.under_sampling import NearMiss

ns = NearMiss(sampling_strategy = 0.8)
X_train_ns,y_train_ns = ns.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes before fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 199022, 1: 342})
The number of classes before fit Counter({0: 427, 1: 342})


In [29]:
from sklearn.ensemble import RandomForestClassifier 

classifier= RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [30]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85287     6]
 [   36   114]]
0.9995084442259752
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85293
           1       0.95      0.76      0.84       150

    accuracy                           1.00     85443
   macro avg       0.97      0.88      0.92     85443
weighted avg       1.00      1.00      1.00     85443



 Never do undersampling unless the dataset is very very small

## Over Sampling

In [31]:
from imblearn.over_sampling import RandomOverSampler

In [39]:
os = RandomOverSampler(sampling_strategy = 0.75)
X_train_os,y_train_os = os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes before fit {}".format(Counter(y_train_os)))

The number of classes before fit Counter({0: 199022, 1: 342})
The number of classes before fit Counter({0: 199022, 1: 149266})


In [40]:
from sklearn.ensemble import RandomForestClassifier 

classifier= RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [41]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85287     6]
 [   35   115]]
0.9995201479348805
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85293
           1       0.95      0.77      0.85       150

    accuracy                           1.00     85443
   macro avg       0.98      0.88      0.92     85443
weighted avg       1.00      1.00      1.00     85443



## SmoteTomek

In [44]:
from imblearn.combine import SMOTETomek

os = SMOTETomek(sampling_strategy = 0.75)
X_train_os,y_train_os = os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes before fit {}".format(Counter(y_train_os)))

The number of classes before fit Counter({0: 199022, 1: 342})
The number of classes before fit Counter({0: 198352, 1: 148596})


In [45]:
from sklearn.ensemble import RandomForestClassifier 

classifier= RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [46]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85287     6]
 [   34   116]]
0.9995318516437859
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85293
           1       0.95      0.77      0.85       150

    accuracy                           1.00     85443
   macro avg       0.98      0.89      0.93     85443
weighted avg       1.00      1.00      1.00     85443



## Ensemble Techniques



In [48]:
from imblearn.ensemble import EasyEnsembleClassifier

In [49]:
easy = EasyEnsembleClassifier()
easy.fit(X_train,y_train)

EasyEnsembleClassifier()

In [ ]:
easy.

In [ ]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))